In [2]:
import stdpopsim
import msprime
import numpy as np
from collections import Counter
from dismal import multimodel, models
from dismal import model_instance
import warnings 
import seaborn as sns
import pandas as pd
import math
import demesdraw

/Users/s2341012/.local/lib/python3.12/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (
/Users/s2341012/.local/lib/python3.12/site-packages/stdpopsim/catalog/HomSap/demographic_models.py:158: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  time=int(extended_GF.time.head(1) - 1), rate=0
/Users/s2341012/.local/lib/python3.12/site-packages/stdpopsim/catalog/HomSap/demographic_models.py:161: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  time=int(extended_GF.time.tail(1) + 1), rate=0


In [3]:
demography = stdpopsim.get_species("DroMel").get_demographic_model("OutOfAfrica_2L06").model

In [4]:
treeseqs = msprime.sim_ancestry(samples={"AFR":2, "EUR":2},
                                       ploidy=1,
                                       demography=demography, 
                                       recombination_rate=2.4e-8, 
                                       sequence_length=100, 
                                       num_replicates=30000)

In [8]:
def seg_sites_from_ts(ts):
    """Add mutations to single treesequence and count number of segregating sites"""
    mts = msprime.sim_mutations(ts, rate=1e-9)
    divmat = mts.divergence_matrix(span_normalise=False)
    state1_s = np.array([divmat[0, 1]])
    state2_s = np.array([divmat[2, 3]])
    state3_s = np.array([divmat[0, 2], divmat[0, 3], 
                             divmat[1, 2], divmat[1, 3]])

    return state1_s, state2_s, state3_s

In [9]:
seg_sites = [seg_sites_from_ts(ts) for ts in treeseqs]

In [20]:
s1 = np.concatenate([entry[0] for entry in seg_sites])
s1

array([1., 0., 0., ..., 1., 0., 0.])

In [22]:
Counter(s1)
s1_arr = np.zeros(500)
for (key, val) in Counter(s1).items():
    s1_arr[(int(key))] = val

In [35]:
demography.migration_matrix

array([[0., 0.],
       [0., 0.]])

In [5]:
sfs = [ts.allele_frequency_spectrum(span_normalise=False) for ts in mut_treeseqs]

In [12]:
sfs_list = [tuple(entry[1:-1]) for entry in sfs]

In [13]:
Counter(sorted(sfs_list)).values()

dict_values([1243, 927, 585, 318, 144, 81, 45, 20, 6, 4, 1, 2386, 17, 1688, 21, 936, 16, 447, 6, 203, 4, 87, 1, 29, 1, 15, 1, 8, 2, 1, 2662, 60, 1943, 37, 962, 24, 447, 13, 191, 7, 82, 2, 40, 5, 11, 1, 2, 5, 2, 1, 2322, 72, 1570, 53, 764, 23, 359, 12, 146, 2, 59, 6, 25, 1, 5, 1, 3, 1648, 53, 1116, 46, 544, 21, 227, 14, 92, 10, 31, 5, 17, 2, 6, 1, 3, 989, 40, 684, 41, 377, 15, 155, 14, 51, 6, 18, 4, 6, 3, 1, 601, 36, 340, 32, 202, 14, 71, 8, 37, 4, 12, 1, 5, 1, 269, 20, 185, 19, 83, 8, 45, 8, 9, 1, 6, 3, 1, 1, 135, 7, 69, 5, 44, 4, 16, 4, 4, 2, 5, 1, 53, 4, 47, 5, 24, 3, 12, 2, 4, 2, 2, 1, 29, 4, 20, 4, 6, 1, 1, 1, 2, 1, 1, 1, 10, 1, 12, 1, 5, 3, 4, 5, 1, 1, 2, 3, 1])

In [14]:
Counter(sorted(sfs_list))

Counter({(2.0, 0.0, 0.0): 2662,
         (1.0, 0.0, 0.0): 2386,
         (3.0, 0.0, 0.0): 2322,
         (2.0, 1.0, 0.0): 1943,
         (1.0, 1.0, 0.0): 1688,
         (4.0, 0.0, 0.0): 1648,
         (3.0, 1.0, 0.0): 1570,
         (0.0, 0.0, 0.0): 1243,
         (4.0, 1.0, 0.0): 1116,
         (5.0, 0.0, 0.0): 989,
         (2.0, 2.0, 0.0): 962,
         (1.0, 2.0, 0.0): 936,
         (0.0, 1.0, 0.0): 927,
         (3.0, 2.0, 0.0): 764,
         (5.0, 1.0, 0.0): 684,
         (6.0, 0.0, 0.0): 601,
         (0.0, 2.0, 0.0): 585,
         (4.0, 2.0, 0.0): 544,
         (1.0, 3.0, 0.0): 447,
         (2.0, 3.0, 0.0): 447,
         (5.0, 2.0, 0.0): 377,
         (3.0, 3.0, 0.0): 359,
         (6.0, 1.0, 0.0): 340,
         (0.0, 3.0, 0.0): 318,
         (7.0, 0.0, 0.0): 269,
         (4.0, 3.0, 0.0): 227,
         (1.0, 4.0, 0.0): 203,
         (6.0, 2.0, 0.0): 202,
         (2.0, 4.0, 0.0): 191,
         (7.0, 1.0, 0.0): 185,
         (5.0, 3.0, 0.0): 155,
         (3.0, 4.0, 0.0): 146,